In [1]:
from lark import Lark
from lark.visitors import Transformer, v_args
from __future__ import annotations

In [2]:
class FieldAndTale:
    """ SQL文を生成するためのクラス  
    parsingした際の各ノード毎にそのノードでの計算を行うSQL文を生成できるようにする.  
    そのノードで二つのSQL文を連結する場合はfieldを結合する(集計関数処理用のstackテーブルがWITHで付いてくるかもなのでwith用テーブルリストも結合)
    そのノードで集計関数処理が必要なら集計用のselect文を生成し,with句でテーブル化,フィールドに追加する(テーブル名を関数が出現した文字の位置で生成)

    field(str): 式を管理(四則演算はfield内で処理できるはず)
    value_table(str): 集計元のデータが入っているテーブル
    group_by(str): 集計単位で使用するフィールド
    withs(list[str]): 集計関数用のWITH句とSELECT文
    
    TODO: ★計算が先か集計が先かを切り替えるフラグも必要
    """
    def __init__(self, field:str, table: str, group_by: str = None, tables=None, withs=[]):
        self.field = field
        self.value_table = table
        self.tables = set([table])
        if tables is not None:
            self.tables = tables
        self.group_by = group_by
        self.withs = withs # 縦横変換用 サブクエリテーブル定義

    def select_to_str(self):
        select = f"SELECT {self.field} AS value FROM {self.value_table}"
        if self.group_by is not None:
            select += f" GROUP BY {self.group_by}"
        return select

    def all_to_str(self):
        withs = ",\n".join(self.withs)
        return f"{withs} \nSELECT {self.field} AS value FROM {','.join(self.tables)};"
    
    def add_unary(self, unary: str) -> FieldAndTale:
        self.field = f"{unary}({self.field})"
        return self

    @staticmethod
    def merge_FieldAndTable(a: FieldAndTale, ope: str, b: FieldAndTale) -> FieldAndTale:
        """2項演算子のFieldAndTableを結合する関数
        a + bなど、2項演算子の処理でFieldAndTableを結合する処理を行う関数

        Args:
            a (FieldAndTale): 1番目のオペランド
            ope (str): 2項演算子
            b (FieldAndTale): 2番目のオペランド

        Returns:
            FieldAndTable: 結合後のFieldAndTable
        """
        field = f"({a.field}){ope}({b.field})"
        tables = a.tables | b.tables
        withs = a.withs + b.withs
        return FieldAndTale(field, a.value_table, a.group_by, tables, withs)

    @staticmethod
    def callFunc(function: str, args: list[FieldAndTale], pos: int, chip_group_by: str) -> FieldAndTale:
        """集計関数呼び出しを行う関数
        引数に渡された複数のFieldAndTableに対して横縦変換を行い,集計関数呼び出しを行う関数
        UNIONで横縦変換するselect文をwitdh句で仮想テーブル化して集計関数を呼び出す

        Args:
            function (str): 呼び出すSQLの関数
            args (list[FieldAndTale]): 関数の引数となるTableAndTable
            post (int): テーブル名生成用関数が現れた文字位置(ユニークであることを想定)
            chip_group_by (str): UNION ALLした後にgroupbyするためのフィールド(chip単位でグルーピングすることを想定)

        Returns:
            FieldAndTale: 関数呼び出し処理をするFieldAndTableオブジェクト
        """
        # 既存のtablesを結合
        tables = set()
        for item in args:
            tables |= item.tables
        # 既存のwiths句を出現順を保持して結合
        withs = [x for item in args for x in item.withs]
        # 引数のFieldAndTableのselect文をwiths句に全部入れてテーブル化
        new_tanles: list[str] = []
        group_by: str = None
        for i, item in enumerate(args):
            new_table = f"with_table_{function}_{pos}_{i}"
            withs += [f"WITH {new_table} AS ({item.select_to_str()})"]
            tables |= set(new_table)
            new_tanles.append(new_table)
            if group_by is None:
                group_by = item.group_by
            else:
                if group_by != item.group_by:
                    # 同じgroup_by同士でないと演算できない(発生しないはず,バグ避け)
                    raise ValueError(f"group_by miss match. {group_by} vs {item.group_by}")
        # 集計用縦積みunion all selectを生成
        new_table = f"with_table_{function}_{pos}"
        # 引数の全テーブルをUNION ALLでつないだテーブルを作る
        select = "\nUNION ALL\n".join([f"SELECT {table}.value AS value FROM {table}" for table in new_tanles])
        new_with = f"WITH {new_table} AS ({select})"
        withs.append(new_with)
        tables |= set(new_table)

        # chip毎の集計を行うselect文を新しいwith句でtable化
        agg_table = f"with_table_{function}_{pos}_agg"
        select = f"SELECT {function}({new_table}.value) AS value FROM {new_table} GROUP BY {chip_group_by}"
        new_with = f"WITH {agg_table} AS ({select})"
        withs.append(new_with)
        tables |= set(agg_table)

        field = f"{agg_table}.value"

        return FieldAndTale(field, agg_table, group_by, set([agg_table]), withs)

In [3]:
# インタプリタの本体
# SQL文に変換する
class ExprTransformer(Transformer):
    def __init__(self, table_name:str, group_by: str = None, chip_group_by: str = None, visit_tokens = True):
        super().__init__(visit_tokens)
        self.table_name = table_name
        self.group_by = group_by
        self.chip_group_by = chip_group_by
    @v_args(meta=True)
    def max(self, meta, args):
        return FieldAndTale.callFunc("MAX", args, meta.column, self.chip_group_by)
    @v_args(meta=True)
    def min(self, meta, args):
        return FieldAndTale.callFunc("MIN", args, meta.column, self.chip_group_by)
    @v_args(meta=True)
    def mean(self, meta, args):
        return FieldAndTale.callFunc("MEAN", args, meta.column, self.chip_group_by)
    @v_args(meta=True)
    def median(self, meta, args):
        return FieldAndTale.callFunc("MEDIAN", args, meta.column, self.chip_group_by)
    def add(self, args):
        # return f"({args[0]}) + ({args[1]})"
        return FieldAndTale.merge_FieldAndTable(args[0], "+", args[1])
    def sub(self, args):
        # return f"({args[0]}) - ({args[1]})"
        return FieldAndTale.merge_FieldAndTable(args[0], "-", args[1])
    def mul(self, args):
        # return f"({args[0]}) * ({args[1]})"
        return FieldAndTale.merge_FieldAndTable(args[0], "*", args[1])
    def div(self, args):
        # return f"({args[0]}) / ({args[1]})"
        return FieldAndTale.merge_FieldAndTable(args[0], "/", args[1])
    def unary_minus(self, args):
        # return f"-({args[0]})"
        return args[0].add_unary("-")
    def unary_plus(self, args):
        # return f"+({args[0]})"
        return args[0].add_unary("+")
    def number(self, args):
        # return str(float(args[0]))
        return FieldAndTale(field=args[0], table=self.table_name)
    def symbol(self, args):
        # return str(args[0])
        # 計測項目については、データが入っているテーブル名を指定
        return FieldAndTale(field=f"{args[0]}.{self.table_name}", table=self.table_name)

In [4]:
args = ["", "sql.txt"]
file_name = args[1]
with open("./testlark.lark", encoding="utf-8") as grammar:
    with open("./"+file_name,encoding="utf-8") as file:
        # text=file.read().replace("\n","").replace(" ","").replace("\t","") # 改行、スペース、タブは排除
        text=file.read()
        # parser = Lark(grammar.read(), parser='lalr', start="expr", transformer=ExprTransformer()) # 式のみ
        # result = parser.parse(text)
        parser = Lark(grammar.read(), parser='lalr', start="expr", propagate_positions=True)
        tree = parser.parse(text)
        # データが入っているテーブル名と集計単位用フィールドを指定して変換
        result = ExprTransformer("hive_table", "wafer_id", "WAFER_ID, GLOBAL_X, GLOBAL_Y").transform(tree)
        print(result.all_to_str())
        print(text)

WITH with_table_MAX_21_0 AS (SELECT TEST_2.hive_table AS value FROM hive_table),
WITH with_table_MAX_21_1 AS (SELECT TEST_3.hive_table AS value FROM hive_table),
WITH with_table_MAX_21 AS (SELECT with_table_MAX_21_0.value AS value FROM with_table_MAX_21_0
UNION ALL
SELECT with_table_MAX_21_1.value AS value FROM with_table_MAX_21_1),
WITH with_table_MAX_21_agg AS (SELECT MAX(with_table_MAX_21.value) AS value FROM with_table_MAX_21 GROUP BY WAFER_ID, GLOBAL_X, GLOBAL_Y),
WITH with_table_MEDIAN_1_0 AS (SELECT (-(1.5))+(TEST_1.hive_table) AS value FROM hive_table),
WITH with_table_MEDIAN_1_1 AS (SELECT with_table_MAX_21_agg.value AS value FROM with_table_MAX_21_agg),
WITH with_table_MEDIAN_1 AS (SELECT with_table_MEDIAN_1_0.value AS value FROM with_table_MEDIAN_1_0
UNION ALL
SELECT with_table_MEDIAN_1_1.value AS value FROM with_table_MEDIAN_1_1),
WITH with_table_MEDIAN_1_agg AS (SELECT MEDIAN(with_table_MEDIAN_1.value) AS value FROM with_table_MEDIAN_1 GROUP BY WAFER_ID, GLOBAL_X, GLOBAL_Y